In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import os.path as path
import itertools
from sklearn.model_selection import train_test_split

In [2]:
import tensorflow.keras as keras
from tensorflow.keras.layers import Input,InputLayer, Dense, Activation, BatchNormalization, Flatten, Conv2D
from tensorflow.keras.layers import MaxPooling2D, Dropout
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import ModelCheckpoint,LearningRateScheduler, \
                                        EarlyStopping
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical, multi_gpu_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
os.environ['CUDA_VISIBLE_DEVICES'] = '3'

In [4]:
data_dir = 'data/'

In [5]:
train_data = np.load(path.join(data_dir, 'imagenet_6_class_train_data.npz'))
val_data = np.load(path.join(data_dir, 'imagenet_6_class_val_data.npz'))
test_data = np.load(path.join(data_dir, 'imagenet_6_class_val_data.npz'))

In [6]:
x_train = train_data['x_data']
y_train = train_data['y_data']
x_val = val_data['x_data']
y_val = val_data['y_data']
x_test = test_data['x_data']
y_test = test_data['y_data']
y_table_array = test_data['y_table_array']

x_train.shape, y_train.shape, x_val.shape, y_val.shape, x_test.shape, y_test.shape, y_table_array.shape

((4680, 341, 341, 3),
 (4680,),
 (1560, 341, 341, 3),
 (1560,),
 (1560, 341, 341, 3),
 (1560,),
 (6, 2))

In [7]:
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)

x_train.shape, y_train.shape, x_val.shape, y_val.shape, x_test.shape, y_test.shape

((4680, 341, 341, 3),
 (4680, 6),
 (1560, 341, 341, 3),
 (1560, 6),
 (1560, 341, 341, 3),
 (1560, 6))

In [8]:
y_list = [text for _, text in y_table_array]
y_list

['bed', 'bird', 'cat', 'dog', 'house', 'tree']

In [9]:
input_shape = x_train[0].shape
output_size = len(y_list)

In [10]:
def build_2d_cnn_custom_ch_128_DO(conv_num=1):
    input_layer = Input(shape=input_shape)
    x = input_layer

    for i in range(conv_num):
        x = Conv2D(kernel_size=5, filters=128*(2**(i//2)), strides=(1,1), padding='same')(x)
#         x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = MaxPooling2D(pool_size=3, strides=(3,3), padding='same')(x)

    x = Flatten()(x)
  
    x = Dropout(0.75)(x)
    output_layer = Dense(output_size, activation='softmax')(x)
    
    model = Model(inputs=input_layer, outputs=output_layer)
    
    return model

In [11]:
for i in range(1, 6):
    model = build_2d_cnn_custom_ch_128_DO(conv_num=i)
    model.summary()
    del model

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 341, 341, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 341, 341, 128)     9728      
_________________________________________________________________
activation (Activation)      (None, 341, 341, 128)     0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 114, 114, 128)     0         
_________________________________________________________________
flatten (Flatten)            (None, 1663488)           0         
_________________________________________________________________
dropout (Dropout)    

In [12]:
data_generator = ImageDataGenerator(
                    rotation_range=30,
                    width_shift_range=0.15,
                    height_shift_range=0.15,
                    shear_range=0.2,
                    zoom_range=0.2,
                    horizontal_flip=True,
                    fill_mode='nearest')

In [13]:
for i in range(3, 6):
    base = 'vis_imagenet_6_class_augmentation_2D_CNN_custom_ch_128_DO_075_DO'
    model_name = base+'_{}_conv'.format(i)
    model = build_2d_cnn_custom_ch_128_DO(conv_num=i)
#     model = multi_gpu_model(model, gpus=2)
    model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=1e-4),
          metrics=['accuracy'])
    model_path = 'model/checkpoint/'+model_name+'_checkpoint/'
    os.makedirs(model_path, exist_ok=True)
    model_filename = model_path+'{epoch:03d}-{val_loss:.4f}.hdf5'
    checkpointer = ModelCheckpoint(filepath = model_filename, monitor = "val_loss", 
                                   verbose=1, save_best_only=True)

    early_stopping = EarlyStopping(monitor='val_loss', patience=100)
    hist = model.fit_generator(
            data_generator.flow(x_train, y_train, batch_size=64),
            steps_per_epoch=len(x_train)//64,
            epochs=10000,
            validation_data=data_generator.flow(x_val, y_val, batch_size=64),
            validation_steps=len(x_val)//64,
            callbacks = [checkpointer, early_stopping],
            workers=4, 
            use_multiprocessing=True
    )

    print()
    print(model_name, 'Model')
    fig, ax = plt.subplots()
    ax.plot(hist.history['loss'], 'y', label='train loss')
    ax.plot(hist.history['val_loss'], 'r', label='val loss')
    ax.plot(hist.history['acc'], 'b', label='train acc')
    ax.plot(hist.history['val_acc'], 'g', label='val acc')
    ax.set_xlabel('epoch')
    ax.set_ylabel('loss')
    ax.legend(loc='upper left')
    plt.show()

    png_path = 'visualization/learning_curve/'
    filename = model_name+'.png'
    os.makedirs(png_path, exist_ok=True)
    fig.savefig(png_path+filename, transparent=True)

    model.save(model_path+'000_last.hdf5')
    del(model)

    model_path = 'model/checkpoint/'+model_name+'_checkpoint/'
    model_filename = model_path + sorted(os.listdir(model_path))[-1]
    model = load_model(model_filename)
    [loss, accuracy] = model.evaluate(x_test, y_test)
    print('Loss:', loss, 'Accuracy:', accuracy)
    print()

    del(model)

Instructions for updating:
Use tf.cast instead.
Epoch 1/1000
25/25 [==============================] - 13s 522ms/step - loss: 1.3519 - acc: 0.4596

Epoch 00001: val_loss improved from inf to 1.35194, saving model to model/checkpoint/vis_imagenet_6_class_augumentation_2D_CNN_custom_ch_128_DO_075_DO_3_conv_checkpoint/001-1.3519.hdf5
74/74 [==============================] - 50s 675ms/step - loss: 1.6047 - acc: 0.3188 - val_loss: 1.3519 - val_acc: 0.4596
Epoch 2/1000
25/25 [==============================] - 13s 512ms/step - loss: 1.2175 - acc: 0.5173

Epoch 00002: val_loss improved from 1.35194 to 1.21746, saving model to model/checkpoint/vis_imagenet_6_class_augumentation_2D_CNN_custom_ch_128_DO_075_DO_3_conv_checkpoint/002-1.2175.hdf5
74/74 [==============================] - 45s 607ms/step - loss: 1.3236 - acc: 0.4667 - val_loss: 1.2175 - val_acc: 0.5173
Epoch 3/1000
25/25 [==============================] - 12s 499ms/step - loss: 1.1590 - acc: 0.5372

Epoch 00003: val_loss improved from 1

Epoch 22/1000
25/25 [==============================] - 12s 495ms/step - loss: 0.7464 - acc: 0.7378

Epoch 00022: val_loss did not improve from 0.72759
74/74 [==============================] - 44s 601ms/step - loss: 0.7292 - acc: 0.7301 - val_loss: 0.7464 - val_acc: 0.7378
Epoch 23/1000
25/25 [==============================] - 13s 502ms/step - loss: 0.7255 - acc: 0.7423

Epoch 00023: val_loss improved from 0.72759 to 0.72552, saving model to model/checkpoint/vis_imagenet_6_class_augumentation_2D_CNN_custom_ch_128_DO_075_DO_3_conv_checkpoint/023-0.7255.hdf5
74/74 [==============================] - 45s 606ms/step - loss: 0.7216 - acc: 0.7419 - val_loss: 0.7255 - val_acc: 0.7423
Epoch 24/1000
25/25 [==============================] - 12s 496ms/step - loss: 0.7366 - acc: 0.7365

Epoch 00024: val_loss did not improve from 0.72552
74/74 [==============================] - 45s 603ms/step - loss: 0.7286 - acc: 0.7453 - val_loss: 0.7366 - val_acc: 0.7365
Epoch 25/1000
25/25 [======================

ResourceExhaustedError: OOM when allocating tensor with shape[64,128,341,341] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node training/Adam/gradients/max_pooling2d_15/MaxPool_grad/MaxPoolGrad}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


In [ ]:
log_dir = 'log'
os.makedirs(log_dir, exist_ok=True)
base = 'vis_imagenet_6_class_augmentation_2D_CNN_custom_ch_128_DO_075_DO'

with open(path.join(log_dir, base), 'w') as log_file:
    for i in range(3, 6):
        model_name = base+'_{}_conv'.format(i)
        print()
        print(model_name, 'Model')
        model_path = 'model/checkpoint/'+model_name+'_checkpoint/'
        model_filename = model_path + sorted(os.listdir(model_path))[-1]

        model = load_model(model_filename)
        model.summary()

        [loss, accuracy] = model.evaluate(x_test, y_test)
        print('Loss:', loss, 'Accuracy:', accuracy)

        del(model)

        log_file.write('\t'.join([model_name, str(accuracy), str(loss)])+'\n')

In [ ]:
for i in range(3, 6):
    model_name = base+'_{}_conv'.format(i)
    print()
    print(model_name, 'Model')
    model_path = 'model/checkpoint/'+model_name+'_checkpoint/'
    model_filename = model_path + '000_last.hdf5'

    model = load_model(model_filename)
    model.summary()

    [loss, accuracy] = model.evaluate(x_test, y_test)
    print('Loss:', loss, 'Accuracy:', accuracy)

    del(model)